In [1]:
import re
import string

import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import TFBertModel, BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
file_name = "data/GloVe/glove.6B.50d.txt"

embedding = {}
idx_to_word = {}
word_to_idx = {}
bad_lines = 0

with open(file_name, 'r', encoding='UTF-8') as f:
    for idx, line in enumerate(f):
        try:
            line = line.strip()
            match_obj = re.match(r'([^\s]+)', line)
            word = match_obj.group(1)
            word_len = len(word)
            word_vec = line[word_len:].replace('\n', '')

            embed = word_vec.strip()
            embed = embed.split()
            embed = np.array(embed, dtype=np.float32)
            
            embedding[idx] = [embed]
            idx_to_word[idx] = word.strip()
            word_to_idx[word.strip()] = idx
        except:
            bad_lines += 1

print(f'Bad lines: {bad_lines}')

Bad lines: 0


In [4]:
w1 = 'banana'
w2 = 'fruit'

w1_embed = embedding[word_to_idx[w1]]
w2_embed = embedding[word_to_idx[w2]]
cosine_similarity(w1_embed, w2_embed)


array([[0.71814054]], dtype=float32)

In [5]:
df = pd.read_csv('data/opted/OPTED-Dictionary.csv')

In [6]:
df.head()

,Word,Count,POS,Definition
0,A,1,"""""","""The first letter of the English and of many o..."
1,A,1,"""""","""The name of the sixth tone in the model major..."
2,A,1,"""""","""An adjective commonly called the indefinite ..."
3,A,1,"""""","""In each; to or for each; as """"""""twenty leagu..."
4,A,1,"""prep.""","""In; on; at; by."""


In [7]:
df["Word"] = df["Word"].str.lower()

In [8]:
df["Word"][100]

'abate'

In [9]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
chars_to_remove = re.compile(r'[{}]'.format(re.escape(string.punctuation)))
df["Definition"] = df["Definition"].map(lambda x: chars_to_remove.sub('', x).lower())

In [11]:
df.head()

,Word,Count,POS,Definition
0,a,1,"""""",the first letter of the english and of many ot...
1,a,1,"""""",the name of the sixth tone in the model major ...
2,a,1,"""""",an adjective commonly called the indefinite a...
3,a,1,"""""",in each to or for each as twenty leagues a da...
4,a,1,"""prep.""",in on at by


In [12]:
print(df["Definition"][3])

in each to or for each as  twenty leagues a day a hundred pounds a year a dollar a yard etc


In [13]:
df.shape

(176009, 4)

In [14]:
df = df[df["Word"].isin(word_to_idx.keys())]

In [15]:
df.shape

(102236, 4)

In [16]:
df = df[df["Word"] != ""]
df = df[df["Definition"] != ""]

In [17]:
df.shape

(102234, 4)

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [19]:
seq_length = 32

def generate_sequences(dataset):
    num_sequences = dataset.shape[0]

    x_token = np.zeros((num_sequences, seq_length))
    x_mask = np.zeros((num_sequences, seq_length))
    y_token = np.zeros((num_sequences, 50), dtype=np.float64)

    for i in range(0, num_sequences):
        try:
            word = dataset["Word"].iloc[i]
            definition = dataset["Definition"].iloc[i]

            tokens = tokenizer.tokenize(definition)
            input = tokenizer.encode_plus(tokens, max_length=seq_length, truncation=True, padding='max_length', add_special_tokens=True, return_tensors='tf')
            
            x_token[i, :] = input['input_ids']
            x_mask[i, :] = input['attention_mask']
            y_token[i, :] = embedding[word_to_idx[word]][0]
        
        except:
            print(f'Error on {i}')


    print(f'Number of sequences: {num_sequences}')

    return x_token, x_mask, y_token, num_sequences

x_token, x_mask, y_token, num_sequences = generate_sequences(df[["Word", "Definition"]])

2021-11-19 13:36:38.166267: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 13:36:38.170620: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 13:36:38.170952: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 13:36:38.192101: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Number of sequences: 102234


In [20]:
dataset = tf.data.Dataset.from_tensor_slices((x_token, x_mask, y_token))
dataset.take(1)

<TakeDataset shapes: ((32,), (32,), (50,)), types: (tf.float64, tf.float64, tf.float64)>

In [21]:
def combine_input_mask(input_id, mask, y_token):
    return {'input_ids': input_id, 'attention_mask': mask}, y_token

dataset = dataset.map(combine_input_mask)
dataset.take(1)

<TakeDataset shapes: ({input_ids: (32,), attention_mask: (32,)}, (50,)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.float64)>

In [22]:
batch_size = 64
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)
dataset.take(1)

<TakeDataset shapes: ({input_ids: (64, 32), attention_mask: (64, 32)}, (64, 50)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.float64)>

In [23]:
bert = TFBertModel.from_pretrained('bert-base-uncased')

2021-11-19 13:37:56.620582: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further tr

In [24]:
input_layer = Input(shape=(seq_length,), name='input_ids', dtype=tf.int32)
mask_layer = Input(shape=(seq_length,),
                   name='attention_mask', dtype=tf.int32)

embedding_layer = bert.bert(input_layer, attention_mask=mask_layer)[1]

out = Dense(1024, activation='relu')(embedding_layer)
out = Dropout(0.2)(out)
out = Dense(50)(out)

model = Model(inputs=[input_layer, mask_layer], outputs=out)

model.layers[2].trainable = False

optimizer = SGD(learning_rate=0.00009, decay=1e-6)
model.compile(optimizer=optimizer,
              loss=tf.losses.mean_squared_error, metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 32)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 32,                                            

In [25]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('reverse_model_whole.h5', monitor='val_loss',
                     mode='min', save_best_only=True, verbose=1)

history = model.fit(dataset, batch_size=64, epochs=100,
                    shuffle=True, verbose=1, callbacks=[es, mc])


Epoch 1/100
1597/1597 [==============================] - 137s 84ms/step - loss: 0.5674 - accuracy: 0.0404
Epoch 2/100
1597/1597 [==============================] - 134s 84ms/step - loss: 0.4910 - accuracy: 0.0960
Epoch 3/100
1597/1597 [==============================] - 135s 84ms/step - loss: 0.4801 - accuracy: 0.1160
Epoch 4/100
1597/1597 [==============================] - 109s 68ms/step - loss: 0.4733 - accuracy: 0.1274
Epoch 5/100
1597/1597 [==============================] - 108s 68ms/step - loss: 0.4681 - accuracy: 0.1339
Epoch 6/100
1597/1597 [==============================] - 109s 68ms/step - loss: 0.4636 - accuracy: 0.1398
Epoch 7/100
1597/1597 [==============================] - 109s 68ms/step - loss: 0.4600 - accuracy: 0.1437
Epoch 8/100
1597/1597 [==============================] - 108s 68ms/step - loss: 0.4566 - accuracy: 0.1494
Epoch 9/100
1597/1597 [==============================] - 109s 68ms/step - loss: 0.4535 - accuracy: 0.1540
Epoch 10/100
1597/1597 [======================

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()